In [ ]:
import albumentations as A
import os
import random
import cv2
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Load images
images_path = "images/hazmat_codes_working"
file_list = os.listdir(images_path)
random.shuffle(file_list)
base_images = [cv2.cvtColor(cv2.imread(os.path.join(images_path, f)), cv2.COLOR_BGR2RGB) 
              for f in file_list[:15]]

random.shuffle(base_images)

# Configure augmentations
import albumentations as A

# Configure extreme augmentations
augmentations = [
    A.RandomRain(
        brightness_coefficient=0.9,  # Darker rain effect
        drop_width=3,                # Thicker rain drops
        drop_length=20,              # Longer rain streaks
        blur_value=5,                # Stronger blur effect
        p=1                          # Always apply
    ),
    A.RandomSunFlare(flare_roi=(0, 0, 1, 1), angle_lower=0.3, p=1),
    A.RandomShadow(
        num_shadows_lower=10,        # More shadows
        num_shadows_upper=15,
        shadow_dimension=8,          # Larger and darker shadows
        shadow_roi=(0, 0, 1, 1),     # Shadows across the entire image
        p=1                          # Always apply
    ),
    A.RandomFog(
        fog_coef_lower=0.8,          # Denser fog
        fog_coef_upper=1.0,
        alpha_coef=0.7,              # Stronger fog opacity
        p=1                          # Always apply
    )
]


# Create figure with structured layout
fig = plt.figure(figsize=(14, 18), dpi=100)
gs = GridSpec(4, 3, figure=fig, 
             width_ratios=[0.2, 1, 1],  # Label column + 2 image columns
             height_ratios=[1, 1, 1, 1],
             wspace=0.02, hspace=0.15)

# Visualization engine
for row_idx in range(4):
    # Label column
    label_ax = fig.add_subplot(gs[row_idx, 0])
    label_ax.text(0.5, 0.5, 
                 f"{augmentations[row_idx].__class__.__name__}",
                 rotation=90,  # Vertical text
                 ha='center', va='center',
                 fontsize=12, fontweight='semibold',
                 fontfamily='serif')
    label_ax.axis('off')

    # Image columns
    orig_ax = fig.add_subplot(gs[row_idx, 1])
    aug_ax = fig.add_subplot(gs[row_idx, 2])
    
    # Apply augmentation
    transformed = A.Compose([augmentations[row_idx]])(image=base_images[row_idx])['image']
    
    # Plot images
    orig_ax.imshow(base_images[row_idx])
    aug_ax.imshow(transformed)
    
    # Column headers (only for first row)
    if row_idx == 0:
        orig_ax.set_title("Original\n", y=1.05, fontsize=12, fontweight='semibold',
                 fontfamily='serif')
        aug_ax.set_title("Augmented\n",y=1.05, fontsize=12, fontweight='semibold',
                 fontfamily='serif')

    # Remove axes
    for ax in [orig_ax, aug_ax]:
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(False)

plt.tight_layout()
plt.show()

In [ ]:
import os
import cv2
import random
import albumentations as A
from PIL import Image
from typing import Union

def generate_augmented_images(
    input_image_path: str,
    output_dir: str,
    augmentation: Union[str, A.Compose],
    num_augmentations: int = 5,
    seed: int = None,
    quality: int = 95,
    prefix: str = "aug",
    verbose: bool = True
) -> list:
    """
    Generate and save augmented images to a specified directory.
    
    Parameters:
    - input_image_path: Path to source image (str)
    - output_dir: Output directory path (str) - will be created if not exists
    - augmentation: Albumentations transform or preset name ('rain', 'fog', etc.)
    - num_augmentations: Number of augmented versions to create (int)
    - seed: Optional random seed for reproducibility (int)
    - quality: Output JPEG quality (1-100)
    - prefix: Filename prefix for output images
    - verbose: Print progress messages
    
    Returns:
    List of saved file paths (list[str])
    """
    # Validate inputs
    if not os.path.isfile(input_image_path):
        raise FileNotFoundError(f"Input image not found: {input_image_path}")
    
    if seed is not None:
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Load image
    image = cv2.cvtColor(cv2.imread(input_image_path), cv2.COLOR_BGR2RGB)
    base_name = os.path.splitext(os.path.basename(input_image_path))[0]
    
    # Define preset augmentations
    augmentation_presets = {
        'rain': A.RandomRain(brightness_coefficient=0.7, drop_width=2, blur_value=3),
        'sun_flare': A.RandomSunFlare(flare_roi=(0, 0, 1, 1), angle_lower=0.3),
        'shadow': A.RandomShadow(num_shadows_lower=1, num_shadows_upper=2, shadow_dimension=4),
        'fog': A.RandomFog(fog_coef_lower=0.5, fog_coef_upper=1, alpha_coef=0.5, p=1)
,
        'snow': A.RandomSnow(brightness_coeff=2.5, snow_point_lower=0.3, snow_point_upper=0.5)
    }
    
    # Configure augmentation pipeline
    if isinstance(augmentation, str):
        if augmentation not in augmentation_presets:
            raise ValueError(f"Unknown preset: {augmentation}. Available: {list(augmentation_presets.keys())}")
        transform = A.Compose([augmentation_presets[augmentation]])
    else:
        transform = augmentation
    
    # Generate augmented images
    saved_paths = []
    for i in range(num_augmentations):
        try:
            augmented = transform(image=image)['image']
            output_path = os.path.join(output_dir, f"{prefix}_{base_name}_{augmentation}_{i}.jpg")
            
            Image.fromarray(augmented).save(
                output_path,
                quality=quality,
                optimize=True,
                subsampling=0  # Keep highest chroma resolution
            )
            
            saved_paths.append(output_path)
            
            if verbose:
                print(f"Saved: {output_path}")
                
        except Exception as e:
            print(f"Error generating augmentation {i+1}: {str(e)}")
    
    return saved_paths

In [ ]:
# get random image

image_path_random = os.path.join(images_path, random.choice(file_list))
print(image_path_random)
# Save a rainy versions of image.jpg
generate_augmented_images(
    input_image_path=image_path_random,
    output_dir="images/augmented",
    augmentation="fog",
    num_augmentations=1,
    seed=42  # Same seed = same augmentations
)

In [ ]:
# get random image
image_path_hold = image_path_random

while image_path_hold == image_path_random:
    image_path_random = os.path.join(images_path, random.choice(file_list))

# Save a rainy versions of image.jpg
generate_augmented_images(
    input_image_path=image_path_random,
    output_dir="images/augmented",
    augmentation="rain",
    num_augmentations=1,
    seed=42  # Same seed = same augmentations
)

In [ ]:
# get random image
image_path_hold = image_path_random

while image_path_hold == image_path_random:
    image_path_random = os.path.join(images_path, random.choice(file_list))

# Save a rainy versions of image.jpg
generate_augmented_images(
    input_image_path=image_path_random,
    output_dir="images/augmented",
    augmentation="shadow",
    num_augmentations=1,
    seed=42  # Same seed = same augmentations
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import string
import os

def plot_images_3_by3(image_paths):
    """Plot 9 images in 3x3 grid without predictions"""
    fig = plt.figure(figsize=(20, 20))
    gs = gridspec.GridSpec(3, 3, width_ratios=[1, 1, 1], 
                          wspace=0.0, hspace=0.2,
                          top=0.95, bottom=0.05, 
                          left=0.05, right=0.95)

    for idx, image_path in enumerate(image_paths[:9]):  # Ensure max 9 images
        row, col = divmod(idx, 3)
        ax = plt.subplot(gs[row, col])
        
        # Simply load and plot image
        image = plt.imread(image_path)
        ax.imshow(image, aspect='auto')
        
        # Add subplot label (a), (b), etc.
        ax.text(0.5, -0.15, f'({string.ascii_lowercase[idx]})',
               size=20, ha='center', transform=ax.transAxes, weight='bold')
        
        # Remove axes
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_axis_off()

    plt.subplots_adjust(wspace=0, hspace=0.3)
    plt.show()

images_path = "data/data_faster_rcnn/test/images"
file_list = os.listdir(images_path)

# add images_path for each image in file list
image_paths = [os.path.join(images_path, file_name) for file_name in file_list]
plot_images_3_by3(image_paths)


In [ ]:
# get random image
image_path_hold = image_path_random

while image_path_hold == image_path_random:
    image_path_random = os.path.join(images_path, random.choice(file_list))

# Save a rainy versions of image.jpg
generate_augmented_images(
    input_image_path=image_path_random,
    output_dir="images/augmented",
    augmentation="sun_flare",
    num_augmentations=1,
    seed=42  # Same seed = same augmentations
)